# Introduction
My cousin Morgan and his wife Lauren are expecting their second child. I asked them what names they are considering. They don't know *what* name they want to give the baby, but they do know what *kind* of name they want to give it. Normal, but not *too* common. You know, distinctive, but not *too* weird. A just-right "Goldilocks" name. 

As Goldilocks's own parents have demonstrated, it's a matter of taste whether a name is TOO common or TOO weird. But I suggested to Morgan that as long as he could put some quantative parameters on too common, he could use naming statistics to generate candidate baby names that are *objectively the *best* names for them. It's almost like SCIENCE! Nerd that I am, I decided to take this tongue in cheek project to the internet. This program gives you the top baby names from Social Security data 1880-2022, but allows you to filter your list a few different ways for what is "too normal." 

So Morg, here's the deal. The initial filters I suggest using here are my best guesses about what kind of names would hit the sweet spot of "normal but not TOO normal" for YOU two specifically. You can adjust the values and rerun this code as many times as you like, of course, but tell me how close I got! Also, if you name your baby by using this code, you are legally obligated to make me a godparent.

# How does this work? 
If you have never used a code notebook before, here are some instructions. A notebook has cells of regular text (like this one) interspersed with cells of code that you can run (like the next one). You can read along, and run the code cells one at a time as you go. To run each cell of code, just select the cell, and click of the play button either directly to the left of it or in the top menu. The output of the code will appear below the cell. Many of them ask you for input--those prompts will appear right below the cell, so you may need to scroll down to see them.

To get started, run the set-up cell directly below this one. You only need to run this once per session, then you can rerun any the other cells as many times as you want.

This cell may take a minute or two to run, so be patient. It will say Done! below it when it is finished. The other cells will be faster.

In [ ]:
#Run this set-up cell to get started

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime

#set display parameter to see full lists of names
pd.options.display.max_rows = None

# Import name data     
names = pd.read_csv('/kaggle/input/us-baby-names-1880-2022-flat-file/allnames.csv',
                   dtype = {'name':str,
                           'sex':str,
                           'count':int,
                           'year':int
                           })
this_year = datetime.date.today().year

print('Done!') 

# The first name was too hot

You don't want a name that's too TRENDY, right? Schools are fully saturated with Emmas and Quinns now. **When you run the next code cell, it will ask you to input a value for an elimination ranking and a value for years.** My guess for what is too trendy for you is anything that's been in the top 10 names for either sex anytime in the last 10 years, so enter 10 for both prompts on this run. (You can run it again later and change all the values.)

In [ ]:
# Get user parameters to eliminate the top ranking names for the last n years
rank = input("Eliminate anything that ranked this number or better: ")
rank = int(rank)
years = input("During the last how many years?: ")
years = int(years)

#Now get those trendy names
trendy = (names[names.year >= this_year-years]
          .groupby(["year", "sex"])
          .apply(lambda x: x.nlargest(rank, "count"))
          .reset_index(drop=True)
          .groupby(['name', 'sex'])
          .size()
          .reset_index()
          .rename(columns={0:'years_on_top'})
          .sort_values(['sex', 'years_on_top'], ascending=[True, False]))
print('Okay, eliminated {} trendy names that made the top {} in the last {} years.'.format(len(trendy), rank, years))
printnames = input('Do you want to see the list? y/n: ')
if printnames == 'y':
    print(trendy)          
else:
    print('Ok, moving on...')


# The next name was overcooked
You probably also don't want a name that's been massively overused by a recent generation, either. I think we have enough Jennifers now. Let's try eliminating anything that has been in the top 10 for at least 3 of the last 20 years. 

So, for the next cell block, enter:
* rank: 10 
* number of times: 3
* years: 20

In [ ]:
#Get user parameters for overused
rank2 = input("Eliminate anything that ranked this number or better: ")
rank2 = int(rank2)
freq2 = input("At least this number of times: ")
freq2 = int(freq2)
years2 = input("During the last how many years?: ")
years2 = int(years2)

#Now get those overused names
overused = (names[names.year >= this_year-years2]
            .groupby(["year", "sex"])
            .apply(lambda x: x.nlargest(rank2, "count"))
            .reset_index(drop=True)
            .groupby(['name', 'sex'])
            .size()
            .reset_index()
            .rename(columns={0:'years_on_top'})
            .sort_values(['sex', 'years_on_top'], ascending=[True, False]))
overused = overused[overused.years_on_top >= freq2]
overused = overused[~overused.name.isin(trendy.name)]
print('Okay, eliminate an additional {} overused names that made the top {} in at least {} of the last {} years.'.format(len(overused), rank2, freq2, years2))
printnames = input('Do you want to see the list? y/n: ')
if printnames == 'y':
    print(overused)          
else:
    print('Ok, moving on...')

# The third name was too cold
There are classics, and then there's creakers. We all know what I'm talking about. John. Mary. The kind of names that barely function, because you have to add a descriptor if you want to specify which person you are talking about: Tall Dave, Cousin Dave, or Bald Dave? 

I'm guessing that threshold for you is something like names that have been in the top ten for ten or more years in the last seventy years. 

So, for the next cell prompt enter:
* rank: 10 
* number of times: 10
* years: 70

In [ ]:
#Get user input on what is too common overall
rank3 = input("Eliminate anything that ranked this number or better: ")
rank3 = int(rank3)
freq3 = input("At least this number of times: ")
freq3 = int(freq3)
years3 = input("During the last how many years?: ")
years3 = int(years3)

#Get those too common names
common = (names[names.year >= this_year-years3]
            .groupby(["year", "sex"])
            .apply(lambda x: x.nlargest(rank3, "count"))
            .reset_index(drop=True)
            .groupby(['name', 'sex'])
            .size()
            .reset_index()
            .rename(columns={0:'years_on_top'})
            .sort_values(['sex', 'years_on_top'], ascending=[True, False]))
common = common[common.years_on_top >= freq3]
common = common[~common.name.isin(trendy.name) & ~common.name.isin(overused.name)]
print('Okay, eliminated {} common names that made the top {} in at least {} of the last {} years.'.format(len(common), rank3, freq3, years3))
printnames = input('Do you want to see the list? y/n: ')
if printnames == 'y':
    print(common)          
else:
    print('Ok, moving on...')

# The last name was just right
Moment of truth! We'll get a list of names that have been popular any time since 1880, minus the ones we just excluded. Let's get anything that's ever been in the top twenty (it's probably fewer than you think!)

So, enter 20 at the next cell prompt.

In [ ]:
rank4 = input('See any names that ranked this number or better: ')
rank4 = int(rank4)

#Get all the other top names
greatest_hits = (names.groupby(["year", "sex"])
             .apply(lambda x: x.nlargest(rank4, "count"))
             .reset_index(drop=True)
             .groupby(['name', 'sex'])
             .size()
             .reset_index()
             .rename(columns={0:'years_on_top'})
             .sort_values(['sex', 'years_on_top'], ascending=[True, False]))
             
#combine all those too common names and remove them from the greatest_hits
too_popular = pd.concat([trendy, overused, common]).drop_duplicates('name').sort_values(['sex','years_on_top'])
greatest_hits = greatest_hits[~greatest_hits.name.isin(too_popular.name)]

print('We found {} names in your Goldilocks zone!'.format(len(greatest_hits)))
printnames = input('Do you want to see the list? y/n: ')
if printnames == 'y':
    print(greatest_hits)          
else:
    print("Hmph! Well try again for a list you DO want to see.")

# Bonus Feature: Ambisexual Names
Morg, remember how I mentioned women with masculine names have been shown to succeed more in STEM and business? Want to see a list of names that have been used for both male and female babies recently? In the entire dataset, there are more than 11,000 names that have been registered for both girls and boys, but most of them are used overwhelmingly for one sex, with just few outliers of the other. Maybe you want to see names where only 5% of the babies cross the gender line, or maybe you would rather see names that are closer to 50-50. So next you will need to specify the level of gender neutrality you want these names to have, by specifying a minimum percentage of time the name should have been used for girls and boys.

Interesting side note: Ambisexual names almost always start as masculine names. When ambisexual names become popular as female name, parents have historically abandoned them as male names. This is why 98% of Ashleys and Leslies are female now, whereas 100% of them were male at the turn of the century, for example. However, I've found at least one recent exceptions to this rule: Shannon has a very long history as ambisexual, but after reaching a low of 4% male in the 90s, Shannon has slowly been regaining ground for males since the 2000s, with male use being 30-42% in the last five years. I wonder if this name is an exception or if there is more of this happening due to recent generations' more flexible attitudes towards gender?

In [ ]:
#Get user input for time frame and gender neutrality
print('Since name gender shifts over time, please specify how many years you want this data to include.')
years_a = input('How many recent years?: ')
years_a = int(years_a)

print('OK, you want names that were used for both boys and girls over the last {} years.'.format(years_a))
f_percent = input('What should the minimum percentage of FEMALE use be in that time? (please enter only a number, no percent sign): ')
f_percent = int(f_percent)
if f_percent >= 1:
    f_percent = f_percent*.01
    
m_percent = input('And what should the minimum percentage of MALE use be? (please enter only a number, no percent sign): ')
m_percent = int(m_percent)
if m_percent >= 1:
    m_percent = m_percent*.01

#Find those ambisexual names!
recent = names[names.year >= this_year-years_a]
recent_by_sex = (recent
                 .groupby(['name', 'sex'])
                 .size()
                 .reset_index())

recent_ambi = set(recent_by_sex[recent_by_sex.duplicated('name')].name)

ambi = (recent[recent.name.isin(recent_ambi)]
               .groupby(["name", "sex"])
               .agg({'year':'count', 'count':'sum'})
               .reset_index()
               .rename(columns={'year':'years_used', 'count':'total'})
               .sort_values(['name', 'sex'], ascending=[True, False])
               .pivot(columns='sex', index='name', values='total'))

ambi['percent_female'] = ambi.F/(ambi.F + ambi.M)
ambi['total_uses'] = ambi.F + ambi.M

popular = ambi[ambi.percent_female >= f_percent]
popular = popular[popular.percent_female <= (1-m_percent)].sort_values('total_uses', ascending = False)
popular.percent_female = popular.percent_female.apply(lambda percent: round(percent, 2))

print('Your criteria returned {} names.'.format(len(popular)))
print('That might be more than you expected, because it includes very rare names.')
abridge = input('Would you like to narrow it down to just the more common names on the list? (y/n): ')
if abridge == 'y':
    satisfied = 'n'
    while satisfied != 'y':
        threshold = input('See only names with at least how many uses? (1000, 5000, etc): ')
        threshold = int(threshold)
        popular = popular[popular.total_uses >= threshold]
        print('There are {} names left now.'.format(len(popular)))
        satisfied = input('Is that list small enough? (y/n): ')
    print(popular)
else:
    seelist = input('Would you like to see the list? (y/n): ')
    if seelist == 'y':
        print(popular)